In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_parquet('../data/raw/train.parquet')
print(df.shape)
df.head()

(5531451, 190)


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,NaN,0,0.000610,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,NaN,0,0.005492,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,NaN,0,0.006986,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,NaN,0,0.006527,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,NaN,0,0.008126,0


In [3]:
def hex_to_int(hex_string):
    last_16_chars = hex_string[-16:]
    return int(last_16_chars, 16)

df['customer_ID'] = df['customer_ID'].apply(hex_to_int).astype('int64')
df['S_2'] = pd.to_datetime(df['S_2'])
df = df.fillna(-127)
# NA data contains signal: it cannot just be dropped
# We impute an integer for compatibility with categirical columns
# We want the value to be out of range for most features
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,-4532153018459703766,2017-03-09,0.938469,0,0.008724,1.006838,0.009228,0.124035,0.0,0.004709,...,-1,-1,-1,0,0,0.0,-127.0,0,0.000610,0
1,-4532153018459703766,2017-04-07,0.936665,0,0.004923,1.000653,0.006151,0.126750,0.0,0.002714,...,-1,-1,-1,0,0,0.0,-127.0,0,0.005492,0
2,-4532153018459703766,2017-05-28,0.954180,3,0.021655,1.009672,0.006815,0.123977,0.0,0.009423,...,-1,-1,-1,0,0,0.0,-127.0,0,0.006986,0
3,-4532153018459703766,2017-06-13,0.960384,0,0.013683,1.002700,0.001373,0.117169,0.0,0.005531,...,-1,-1,-1,0,0,0.0,-127.0,0,0.006527,0
4,-4532153018459703766,2017-07-16,0.947248,0,0.015193,1.000727,0.007605,0.117325,0.0,0.009312,...,-1,-1,-1,0,0,0.0,-127.0,0,0.008126,0


In [4]:
''' Feature aggregation based on https://www.kaggle.com/code/cdeotte/xgboost-starter-0-793, too expensive for data augmentation.
def process_and_feature_engineer(df):

    cs = [c for c in list(df.columns) if c not in ['customer_ID', 'S_2']]
    cat_features = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'D_87']
    num_features = [c for c in cs if c not in cat_features]

    # Careful: std assumes 1 dof and will return NaN if just one value
    test_num_agg = df.groupby('customer_ID')[num_features].agg(['mean', 'std', 'min', 'max', 'last']).fillna(0)
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby('customer_ID')[cat_features].agg(['last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg    
    return df

features = process_and_feature_engineer(df)
print(features.shape)
'''

most_recent_indices = df.groupby('customer_ID')['S_2'].idxmax()
features = df.loc[most_recent_indices]
features = features.set_index('customer_ID')
print(features.shape)

(458913, 189)


In [5]:
targets = pd.read_csv('../data/raw/train_labels.csv')
targets['customer_ID'] = targets['customer_ID'].apply(hex_to_int).astype('int64')
targets = targets.set_index('customer_ID')
train_data = features.merge(targets, left_index=True, right_index=True, how='left')
train_data['target'] = train_data['target'].astype('int8')
print(train_data.shape)
train_data.head()

(458913, 190)


,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
-9223358381327749917,2018-03-12,0.387708,0,0.460779,0.073879,0.500263,0.155165,0.0,0.610640,-127.0,...,-1,-1,0,0,0.0,-127.0,0,0.006902,0,1
-9223193039457028513,2018-03-10,1.001372,0,0.006264,0.815078,0.001226,-127.000000,0.0,0.002803,-127.0,...,-1,-1,0,0,0.0,-127.0,0,0.000551,0,0
-9223189665817919541,2018-03-03,0.694073,0,0.010064,0.810628,0.009023,-127.000000,0.0,0.004238,-127.0,...,-1,-1,0,0,0.0,-127.0,0,0.003858,0,0
-9223188534444851899,2018-03-27,0.787945,0,0.002275,0.819601,0.009578,-127.000000,0.0,0.009366,-127.0,...,-1,-1,0,0,0.0,-127.0,0,0.000559,0,0
-9223173911659837606,2018-03-10,0.040486,13,0.113040,0.037854,0.006434,0.283160,0.0,0.588810,-127.0,...,-1,-1,0,0,0.0,-127.0,0,0.007221,0,1


In [6]:
from sklearn.model_selection import train_test_split

train_data.to_csv('../data/processed/train.csv')
X = train_data.drop(columns=['S_2', 'target']).values
y = train_data['target'].values

''' In practice, categorical columns have minimal impact on model performance.
feature_cols = list(X.columns)
cat_features = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'D_87']
num_features = [c for c in feature_cols if c not in cat_features]
X = X[num_features]
'''

# 50000 for model validation, 50000 for data valuation, rest for training
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=100000, random_state=42, stratify=y)
X_test, X_v, y_test, y_v = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)

np.savez('../data/processed/train', x=X_train, y=y_train)
np.savez('../data/processed/test', x=X_test, y=y_test)
np.savez('../data/processed/v', x=X_v, y=y_v)